In [1]:
"""PageRank Algorithm Python accessing files stored in AWS S3 Storage

The script is an example given by Apache Spark for the PageRank algorithm.
This script requires that pandas, pysaprk, boto3 packages be installed within the Python environment. 

Note: Make sure to upload the twitter_combined.txt to your S3 bucket and assign the bucket name and the file name.

"""

In [ ]:
import boto3	
import numpy as np
import pandas as pd
import dask.dataframe as dd
from io import StringIO, BytesIO
import re
from pyspark.sql import SparkSession
from operator import add

# Connect to AWS S3 and assign the bucket_name and specify the file name

s3 = boto3.client("s3")
s3_resource = boto3.resource('s3')


# TODO: Specify the bucket name where your file is stored
bucket_name = "mandili" 

# TODO: Specify the file name
obj = s3.get_object(Bucket=bucket_name, Key="twitter_combined.txt")
df = pd.read_csv(BytesIO(obj['Body'].read())

In [3]:
# This is the entry point to programming Spark with the Dataset and DataFrame API.
spark = SparkSession.builder.appName("PythonPageRank").getOrCreate()

# Create a spark dataframe from a pandas dataframe
df = spark.createDataFrame(df)

In [4]:
def computeContribs(urls, rank):
    """Calculates the URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)
        
def parseNeighbors(urls):
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

In [5]:
# Load in the input file, we will access the input from a spark dataframe here
#lines = spark.read.text('path/to/file').rdd.map(lambda r: r[0])
lines = df.rdd.map(lambda r: r[0])

# Loads all URLs from input file and initialize their neighbors
links = lines.map(lambda line : parseNeighbors(line)).distinct().groupByKey().mapValues(lambda x: list(x)).cache()

# Loads all URLs with other URL(s) link to from input file and initialize ranks of them to one
ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))

# Calculates and updates URL ranks continuously using PageRank algorithm
for iteration in range(int(10)):
    
    # Calculates URL contributions to the rank of other URLs
    contribs = links.join(ranks).flatMap(lambda url_urls_rank: computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))
    
    # Re-calculates URL ranks based on neighbor contributions
    ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)
    
# Collects all URL ranks
for (link, rank) in ranks.collect():
    print("%s has rank: %s." % (link, rank))

92336981 has rank: 1.9430573040299834.
148745060 has rank: 2.748661895942804.
17654192 has rank: 3.5523979676709656.
163090431 has rank: 0.9654809080361126.
359757400 has rank: 0.7244978910054823.
22949668 has rank: 1.5732146468823758.
36767299 has rank: 6.078044315244232.
333021748 has rank: 0.5240168644292261.
74580436 has rank: 9.161264885814727.
109055708 has rank: 0.243633636313562.
50059478 has rank: 0.2558339639007986.
40470589 has rank: 0.3860862103681646.
55387080 has rank: 0.9814977375843512.
160664744 has rank: 0.26487427940525843.
125120339 has rank: 1.385268029102299.
128260873 has rank: 1.7432685988800662.
269304313 has rank: 0.7868989691767964.
232561503 has rank: 1.0556397032771976.
361142565 has rank: 3.311248542445357.
282679866 has rank: 0.40841315581781557.
93069445 has rank: 0.8437095625979838.
73738774 has rank: 0.6231924700089181.
120599128 has rank: 0.36771540729523683.
17434613 has rank: 2.7436156800215286.
160565602 has rank: 0.6822976765525969.
114873228 has 

233631354 has rank: 0.8005862381744108.
24946566 has rank: 0.26001163676892314.
40366279 has rank: 0.26709464007416406.
213867225 has rank: 0.38823105798402935.
376115787 has rank: 0.26709464007416406.
15627593 has rank: 0.1647478310697821.
15667851 has rank: 0.1984709192485007.
104399036 has rank: 0.1647478310697821.
217862436 has rank: 0.20656786237222674.
329399970 has rank: 0.15241792181222952.
272637010 has rank: 0.15241792181222952.
112550509 has rank: 0.4981328557679999.
118584725 has rank: 0.3623042657840968.
17925183 has rank: 0.3035831853025427.
40718256 has rank: 0.32687232575187986.
30325385 has rank: 0.18387560075588413.
252575365 has rank: 0.29606547921005033.
26518842 has rank: 0.2573892140666054.
19162458 has rank: 0.3803295219777847.
153093991 has rank: 0.3089944038218908.
23419059 has rank: 0.1899792086160462.
261930465 has rank: 0.32709400966743474.
408690110 has rank: 0.40736259716427037.
14148403 has rank: 0.2416080644900885.
120232348 has rank: 0.20996732608052646

146978684 has rank: 0.5042324561103002.
28023266 has rank: 0.6772114111561226.
452385699 has rank: 0.4083137635114147.
17249855 has rank: 0.4102381774060604.
17916175 has rank: 0.8634949432517046.
151145458 has rank: 0.5882391106310302.
14116619 has rank: 0.4195874633161355.
256801056 has rank: 1.7210278599706403.
2733051 has rank: 0.2963865573669615.
17976777 has rank: 0.3403735857127598.
44442585 has rank: 0.346643578322809.
25302558 has rank: 0.15137849526616234.
66251141 has rank: 0.2299079284830416.
57093688 has rank: 0.21971387777870222.
108361327 has rank: 1.010464122561796.
49689684 has rank: 0.30677705792499943.
133956377 has rank: 0.8308218933397966.
22923478 has rank: 1.0634991526692952.
23673139 has rank: 0.6904101152009305.
211188770 has rank: 0.44072984156394834.
20959983 has rank: 0.21639914357451245.
106933548 has rank: 0.25359198036847946.
123862119 has rank: 0.8122071052556921.
94583129 has rank: 0.2988023691804499.
86227873 has rank: 0.2177402894945115.
325236758 has

317932183 has rank: 0.5612729303953218.
318549400 has rank: 0.5767164739348599.
16594064 has rank: 1.3010090661381295.
14255502 has rank: 0.34588973409724766.
84189122 has rank: 0.18668118114582602.
80798215 has rank: 0.47670674724088125.
252429764 has rank: 0.2334050211078197.
19223996 has rank: 1.733646509294281.
19997104 has rank: 1.242201058588927.
109550381 has rank: 0.7597317450879502.
17832434 has rank: 0.5819785235768808.
27934142 has rank: 2.7915988726525787.
22944753 has rank: 1.042150945579778.
487602166 has rank: 0.8247811322317941.
64415248 has rank: 0.214252405476334.
263293212 has rank: 0.25112203663164046.
28118794 has rank: 0.2871582163981224.
167164816 has rank: 0.3711183383394165.
326259198 has rank: 0.6641671515083899.
140757844 has rank: 0.16593341521911037.
385959631 has rank: 0.16593341521911037.
278181157 has rank: 0.7259894637731019.
411011857 has rank: 1.4833702483671083.
242725672 has rank: 1.8904898184096641.
191897977 has rank: 0.24022622751831543.
18485795

10146102 has rank: 1.702814792351259.
410045750 has rank: 0.8232813050249104.
81229871 has rank: 1.5940163998852375.
14587578 has rank: 3.989299875685872.
385154183 has rank: 1.4302871833657438.
15518750 has rank: 0.3102540353976625.
19084990 has rank: 2.6451681979766515.
92591481 has rank: 3.9059008150231334.
172959279 has rank: 1.4634640251607474.
352226153 has rank: 1.262275578180783.
31353077 has rank: 49.30730827352538.
248044499 has rank: 3.1181394149554515.
140541581 has rank: 0.39835148276687415.
14619878 has rank: 1.242063173937017.
38514229 has rank: 3.822513861412072.
21446327 has rank: 0.5876306615252301.
245396123 has rank: 2.058321528718556.
20751546 has rank: 0.7320620431513595.
85740936 has rank: 0.5205559667708651.
16043008 has rank: 1.6583739790545924.
8005202 has rank: 1.140836819535275.
49629418 has rank: 2.4092411633494377.
25106800 has rank: 2.9400348767774336.
13185852 has rank: 5.176596829480881.
15657398 has rank: 1.7516789978374987.
102903198 has rank: 1.76807

...

17719233 has rank: 1.254754718482462.
19568273 has rank: 0.48982735608377526.
42912024 has rank: 1.3184705666131293.
197533538 has rank: 0.35538620505438834.
22602849 has rank: 0.7616286601850198.
499839056 has rank: 0.3177495971921118.
217236323 has rank: 0.2822535584157314.
113677103 has rank: 0.1934502321972858.
14277302 has rank: 0.2289777165504865.
60680464 has rank: 0.18576549545584153.
191372457 has rank: 0.46415828663817127.
132022097 has rank: 1.518447319219918.
5688732 has rank: 0.638682672599177.
18369105 has rank: 0.32666826840890056.
14401638 has rank: 0.23798768887418137.
20658373 has rank: 0.3099195103020841.
176686379 has rank: 0.5106954907801338.
62431223 has rank: 0.5553187595596399.
310651459 has rank: 0.36653062327367714.
19391595 has rank: 0.7551842080475812.
107438572 has rank: 0.24709841320331755.
275575393 has rank: 1.1956213523395123.
134103776 has rank: 0.298315548900985.
98959541 has rank: 0.22098016722242678.
22790158 has rank: 1.1410643763519177.
16602705 h

20424865 has rank: 0.7543202633770192.
15205995 has rank: 0.7001027334705585.
20360524 has rank: 2.3776550016838263.
346209425 has rank: 0.8807892087743541.
14561327 has rank: 4.674593232155358.
14245846 has rank: 1.3994604086991305.
14936051 has rank: 0.7418030560271179.
15220205 has rank: 2.8299989898553504.
115815506 has rank: 0.4391791600705428.
28192145 has rank: 2.4816710922510503.
1608991 has rank: 0.9932724754459612.
209806284 has rank: 1.06963694674561.
84193657 has rank: 0.8517412521604233.
16736535 has rank: 4.608386516043892.
443726687 has rank: 0.2816584405153632.
17296523 has rank: 1.8573147833481656.
182159386 has rank: 1.1893108532351768.
382641626 has rank: 0.2379714195525019.
186632886 has rank: 0.32462533456374165.
270111468 has rank: 0.2983672535596642.
301123882 has rank: 0.2664035031846983.
277033463 has rank: 0.425511901030445.
138794168 has rank: 1.09643615491806.
156588288 has rank: 1.9731613621660689.
42096654 has rank: 0.9045645474245961.
14584644 has rank: 0

310533612 has rank: 0.15881029748907174.
291797030 has rank: 0.23125700426727547.
27922297 has rank: 0.1876611238536617.
111323990 has rank: 0.3084498544140406.
532798407 has rank: 0.17431317537030427.
18797870 has rank: 0.3038676113393586.
16412992 has rank: 0.22469363663040592.
496552493 has rank: 0.16367222179161522.
541864710 has rank: 0.16367222179161522.
93461826 has rank: 0.17720058701702268.
134916751 has rank: 0.15253164643083744.
81407491 has rank: 0.19172302675842445.
15584455 has rank: 0.15201204659448783.
140620887 has rank: 0.15557055588506577.
277220771 has rank: 0.1799515187673719.
15381737 has rank: 0.2414796771526037.
38947371 has rank: 0.1924642162711368.
556212490 has rank: 0.2657349869450859.
155355874 has rank: 0.21534420058700465.
100560313 has rank: 0.1640573874417547.
14372818 has rank: 0.16173342320571565.
24799500 has rank: 0.15299586141102403.
207427174 has rank: 0.15299586141102403.
282743175 has rank: 0.3119037684147996.
54498789 has rank: 0.15233423980945

155393003 has rank: 0.15133289685649698.
144301261 has rank: 0.15133289685649698.
480442071 has rank: 0.15133289685649698.
286277187 has rank: 0.37552819006303806.
197176358 has rank: 0.20006050482213206.
235755424 has rank: 0.3375867528300066.
346126683 has rank: 0.23731095592565682.
18065572 has rank: 0.3930761256174739.
466870004 has rank: 0.5164250119394629.
166991113 has rank: 0.20179512484224268.
123100479 has rank: 0.46932176807295667.
344756151 has rank: 0.19388996691359778.
60656644 has rank: 0.6745910183328768.
270343681 has rank: 0.16460235587029995.
6632072 has rank: 0.16460235587029995.
126010178 has rank: 0.16460235587029995.
195530319 has rank: 0.23210520935857704.
56919090 has rank: 0.34515535459962826.
177229224 has rank: 0.8167073817711702.
16317965 has rank: 0.447570457303785.
18788297 has rank: 0.16583825789847653.
16336509 has rank: 0.2916944849334101.
40016838 has rank: 0.30797886953608594.
22737163 has rank: 0.2802470680619334.
71583567 has rank: 0.44152157979121

160614743 has rank: 0.18536967681961328.
76057704 has rank: 0.25874655906840305.
5924082 has rank: 0.5484057666972273.
15931534 has rank: 1.4807151154320664.
40225021 has rank: 0.23958538687667916.
331646683 has rank: 0.49195193614424804.
8841372 has rank: 0.6560836515464296.
20490187 has rank: 0.3604749670245537.
24332709 has rank: 1.4247449337191453.
105001809 has rank: 0.9478370980741647.
259365099 has rank: 0.27511627244819803.
35327613 has rank: 0.15376937327089377.
258142292 has rank: 0.5866114229830072.
212939136 has rank: 0.2580008620048232.
1429761 has rank: 0.9850070709494819.
261699092 has rank: 0.2339622004586988.
277241120 has rank: 0.17420573122297284.
232684632 has rank: 0.4313534324034055.
17624624 has rank: 0.173633162665628.
42251044 has rank: 0.27218746897425744.
138233127 has rank: 0.25596870471172817.
177540789 has rank: 1.0114264804072848.
177820708 has rank: 0.3333664293533894.
19244378 has rank: 0.15294712315140765.
20993292 has rank: 1.127257532401253.
49902972

In [6]:
spark.stop()